In [1]:
!pip install pyspark findspark

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 17.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=e298e93d24987392e38dabd2d7942e08b319855d7a6e53b348b7beffa357df41
  Stored in directory: /root/.cache/pip/wheels/07/fb/67/b9f2c0242d156eaa136b45ae4fd99d3e7c0ecc2acfd26f47b9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 07:54:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Создание DataFrame**

Из RDD

In [3]:
def cleaning(row):
    row = row.split('\t')[:3]
    row = [float(val) for val in row]
    return row

In [5]:
ratings = sc.textFile('/kaggle/input/userddd/user_ratedmovies.dat')

first_row = ratings.first()
ratings = ratings.filter(lambda row: row != first_row)\
                 .map(cleaning)

In [6]:
columns = first_row.split('\t')[:3]

In [7]:
columns

['userID', 'movieID', 'rating']

In [8]:
ratings.take(5)

[[75.0, 3.0, 1.0],
 [75.0, 32.0, 4.5],
 [75.0, 110.0, 4.0],
 [75.0, 160.0, 2.0],
 [75.0, 163.0, 4.0]]

In [9]:
df_rdd = spark.createDataFrame(ratings, columns)

In [10]:
df_rdd

DataFrame[userID: double, movieID: double, rating: double]

Можно еще вот так:

In [11]:
df_rdd = ratings.toDF(columns)

In [12]:
df_rdd

DataFrame[userID: double, movieID: double, rating: double]

Так, а если не хочу вот эти приседания с RDD, а хочу сразу из файла?

In [15]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .load("/kaggle/input/userddd/user_ratedmovies.dat")

In [16]:
df

DataFrame[userID: string, movieID: string, rating: string, date_day: string, date_month: string, date_year: string, date_hour: string, date_minute: string, date_second: string]

Все в string, так не пойдет, давайте автоматически определим тип данных

In [18]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true', 'inferSchema': 'true'})\
          .load("/kaggle/input/userddd/user_ratedmovies.dat")

In [19]:
df

DataFrame[userID: int, movieID: int, rating: double, date_day: int, date_month: int, date_year: int, date_hour: int, date_minute: int, date_second: int]

In [20]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

А можно заранее сказать, какой тип данных я ожидаю?

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [22]:
schema = StructType([ \
    StructField("userID",IntegerType(),True), \
    StructField("movieID",IntegerType(),True), \
    StructField("rating",DoubleType(),True), \
    StructField("date_day", StringType(), True), \
    StructField("date_month", StringType(), True), \
    StructField("date_year", IntegerType(), True), \
    StructField("date_hour", IntegerType(), True), \
    StructField("date_minute", IntegerType(), True), \
    StructField("date_second", IntegerType(), True)
  ])

In [24]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .schema(schema)\
          .load("/kaggle/input/userddd/user_ratedmovies.dat")

In [ ]:
df.printSchema()

In [ ]:
df.show(10)

Но есть уже готовая обертка под все нужды

In [ ]:
df = spark.read.csv(path='user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [ ]:
df.printSchema()

Так, а как сохранить? Лучше быть аккуратнее с overwrite, перезапишет весь указанный путь, append будет безопаснее

In [ ]:
df.write.option("header",True)\
        .mode("overwrite")\
        .parquet('write_1.parquet')

А что с партицированием?

In [ ]:
df.write.option("header",True)\
        .partitionBy('date_year')\
        .mode("overwrite")\
        .parquet('write_2.parquet')

Кстати, раз уж заговорили про схемы данных, то их можно задавать интереснее, например, под группированные данные

In [ ]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

Со структурой можно работать и менять ее под ваши нужны

In [ ]:
updatedDF = df2.withColumn("OtherInfo", 
    F.struct(F.col("id").alias("identifier"),
    F.col("gender").alias("gender"),
    F.col("salary").alias("salary"),
    F.when(F.col("salary").cast(IntegerType()) < 2000,"Low")
      .when(F.col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

Что мы там сделали????

1) Создали новую структуру данных OtherInfo

2) Передали туда id (переименовав столбец), gender, salary

3) Создали столбец Salary_grade из условий

4) удалили id, gender, salary из старой структуры

Есть и еще структуры данных!

In [6]:
from pyspark.sql.types import ArrayType, MapType

In [ ]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

In [ ]:
structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

In [ ]:
df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

In [ ]:
df3.select('properties').collect()

**Описание данных**

Общее описание данных

In [ ]:
df = spark.read.csv(path='user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [ ]:
df.describe().show()

In [ ]:
df.summary().show()

Количество записей

In [ ]:
df.count()

Количество партиций

In [ ]:
df.rdd.getNumPartitions()

 Менять число партиций можно, все как с rdd

In [ ]:
df = df.repartition(5)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df = df.coalesce(2)

In [ ]:
df.rdd.getNumPartitions()

**Различные методы**

Ну теперь давайте тыкать 

Удаляем дубликаты и помним, что есть actions и transformations, count заставит все сделать

In [ ]:
df_without_duplicates = df.drop_duplicates()

Есть alias

In [ ]:
df_without_duplicates = df.dropDuplicates()

Как удалить дубликаты по отдельным колонкам?

In [ ]:
df_without_duplicates = df.drop_duplicates(['userID', 'rating'])

In [ ]:
df_without_duplicates.count()

In [ ]:
df_without_duplicates.show(10)

Корреляции

In [ ]:
df.corr('rating', 'date_day')

In [ ]:
df.corr('rating', 'date_hour')

In [ ]:
df.corr('rating', 'date_year')

Как закинуть данные в любимый pandas?

 Самый простой вариант - встроенный метод, но он жутко медленный при существенном объеме данных. Лучше сохранить паркет и считать через pd.read_parquet()

In [7]:
import pandas as pd

In [63]:
%%timeit 

pandas_df = df.toPandas()

KeyboardInterrupt: 

In [ ]:
pandas_df = df.toPandas()
pandas_df

In [ ]:
pandas_df.info()

In [ ]:
%%timeit 

pandas_df = pd.read_parquet('write_1.parquet')

In [ ]:
pandas_df = pd.read_parquet('write_1.parquet')
pandas_df

In [ ]:
pandas_df.info()

Как говорили на лекции, может все упасть например тут (когда делаем toPandas). Как перейти к итератору?

prefetchPartitions - подготавливать ли следующую партию данных, пока не запросили

In [ ]:
iter_df = df.toLocalIterator()

In [ ]:
row = iter_df.send(None)

In [ ]:
row

In [ ]:
row.asDict()

Отсюда идея: можно вытягивать данные по 1 записи и записывать в датафрейм. Долго, но зато отработает.

In [ ]:
iter_df = df.toLocalIterator()

In [ ]:
list_of_rows = [value for value in iter_df]
print(len(list_of_rows))

In [ ]:
df.columns

In [ ]:
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

In [ ]:
%%timeit

iter_df = df.toLocalIterator()
list_of_rows = [value for value in iter_df]
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

In [ ]:
pandas_df

In [ ]:
pandas_df.info()

**Show**

In [ ]:
df.show(10)

Обрезаем до 2 символов

In [ ]:
df.show(10, truncate=2)

вертикальное отображение

In [ ]:
df.show(10, vertical=True)

**Select**

В PySpark функция select() используется для выбора одного, нескольких столбцов по индексу, всех столбцов из списка и вложенных столбцов из фрейма данных. Функция PySpark select() является функцией преобразования, поэтому она возвращает новый фрейм данных с выбранными столбцами.

In [ ]:
df.columns

Упс, pandas-style тут не приветствуется

In [ ]:
df.userID.show(5)

In [ ]:
df.select('userID').show(5)

Куча вариантов, выбирайте любой

In [ ]:
df.select('userID', 'rating').show(5)

In [ ]:
df.select(['userID', 'rating']).show(5)

In [ ]:
df.select(df.userID,df.rating).show(5)

In [ ]:
df.select(df['userID'],df['rating']).show(5)

In [ ]:
df.select(F.col("userID"), F.col("rating")).show(5)

можно налету переименовать столбец

In [ ]:
df.select(df.userID, df.rating.alias('mark')).show(5)

In [ ]:
#регулярки
df.select(df.colRegex("`d+.*y`")).show(5)

примеры с другим датафреймом, где структура сложнее

In [ ]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

In [ ]:
df2.select('name').show(5)

In [ ]:
df2.select('name.lastname').show(5)

In [ ]:
df2.select('name.firstname', 'name.lastname').show(5)

**withColumn**

PySpark withColumn() - это функция преобразования (transform), которая используется для изменения значения, преобразования типа данных существующего столбца, создания нового столбца и многого другого. Поговорим о часто используемых операциях со столбцами данных PySpark, используя примеры.

In [ ]:
df.printSchema()

Меняем тип данных

In [ ]:
df.withColumn("date_month", F.col("date_month").cast("String")).printSchema()

Модифицировать столбец/создать новый

In [ ]:
df.withColumn("rating_x_10",F.col("rating") * 10).show(5)

Делаем 2 константных столбца

In [ ]:
df.withColumn('fix_1', F.lit(1)).withColumn('fix_2', F.lit(2)).show(5)

**withColumnsRenamed**

Предыдущий вариант не давал возможности переименовать столбцы, это можно сделать иначе

In [ ]:
df.withColumnRenamed('rating', 'mark').show(5)

**filter (where) и иные филтрации**

Функция PySpark filter() используется для фильтрации строк из RDD / DataFrame на основе заданного условия или выражения SQL, вы также можете использовать предложение where() вместо filter() обе эти функции работают аналогично.

1 условие

In [ ]:
df.filter(df.rating == 5.0).show(5)

In [ ]:
df.filter(~(df.rating == 5.0)).show(5)

In [ ]:
df.filter('rating = 5').show(5)

Несколько условий

In [ ]:
df.filter((df.rating == 5.0) & (df.date_year == 2006)).show(5)

In [ ]:
df.filter('(rating = 5.0) and (date_year = 2006)').show(5)

In [ ]:
df.filter('(rating = 5.0) and (userID between 70 and 80)').show(5)

фильтр по списку значений из list

In [ ]:
years = [2006, 2007]

In [ ]:
df.filter(df.date_year.isin(years)).show(5)

 проверим

In [ ]:
df.filter(df.date_year.isin(years))\
  .select('date_year')\
  .dropDuplicates()\
  .collect()

создадим игрушечный датайфрейм для текстовых столбцов

In [ ]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5,"Rames Black"), (6, 'Albus Torch'),
     (7, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

In [ ]:
df2.show()

In [ ]:
df2.filter('name like "R%"').show()

In [ ]:
df2.filter(df2.name.startswith('R')).show()

In [ ]:
df2.filter(df2.name.endswith('Tf')).show()

In [ ]:
df2.filter(df2.name.contains('Wil')).show()

Бывает, что у нас внутри датафрейма есть массив и с ним что-то хочется сделать

In [ ]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

In [ ]:
df3.filter(F.array_contains(df3.hobbies,"football")) \
    .show()  

**Сортировка**

сделаем еще фильтрацию, чтобы увидеть результат (orderBy тут аналог)

In [ ]:
df.filter(df.userID == 75).sort(df.date_minute, df.rating.desc()).show(20)

**groupby**

Когда мы выполняем groupBy() в PySpark DataFrame, он возвращает объект GroupedData, который содержит следующие агрегатные функции:

min(), max(), mean(), count(), sum(), avg(), agg(), pivot() 

In [ ]:
df.groupby('date_year').mean('rating').collect()

мы уже умеем применять разные методы

In [ ]:
df.groupby('date_year')\
  .mean('rating')\
  .sort('date_year')\
  .collect()

In [ ]:
df.filter(df.rating <= 2)\
  .groupby('date_year')\
  .count()\
  .withColumnRenamed('count', 'number')\
  .sort('date_year')\
  .collect()

несколько колонок

In [ ]:
df.groupBy("date_year", "date_month") \
  .mean("rating", "userID") \
  .sort('date_year', 'date_month') \
  .show()

Для того, чтобы делать несколько разных агрегаций и еще менять сразу имя столбца нужно немного изменить синтаксис

И стоит помнить, что персентили тут считаются приближенно

In [ ]:
df.groupBy("date_year") \
    .agg(F.min("rating").alias("min_rating"), \
         F.mean("rating").alias("mean_rating"), \
         F.max("rating").alias("max_rating"),
         F.percentile_approx("rating", 0.5).alias("median")
         ) \
    .show()

Еще можно сделать pivot

In [ ]:
df.groupBy('date_year')\
  .pivot('date_month')\
  .mean('rating')\
  .show(5)

**Join's**

Куда же без них. Что есть: INNER, LEFT OUTER, RIGHT OUTER, LEFT ANTI, LEFT SEMI, CROSS, SELF JOIN

Благодаря оптимизации в датафреймах уже все хорошо работает, спасибо catalist, но чудеса не вечны и плохой код/незнание данных все равно даст о себе знать

Сделаем для себя несколько таблиц, чтобы можно было экспериментировать

In [ ]:
df_mean_user_rating = df.groupBy("userID")\
                        .mean('rating')\
                        .withColumnRenamed('avg(rating)', 'avg_rating_all')

df_mean_user_rating_year = df.groupby('userID', 'date_year')\
                             .mean('rating')\
                             .withColumnRenamed('avg(rating)', 'avg_rating_year')

In [ ]:
df_mean_user_rating.printSchema()

df_mean_user_rating_year.printSchema()

И давайте все в 1 блоке кода, чтобы не растягивать

In [ ]:
df.join(df_mean_user_rating, on=df.userID == df_mean_user_rating.userID, how='inner')\
  .join(df_mean_user_rating_year, on=[df.userID == df_mean_user_rating_year.userID,
                                      df.date_year == df_mean_user_rating_year.date_year],
        how='inner')

Надо удалить дублирующиеся столбцы

In [ ]:
res_join = df.alias('t').join(df_mean_user_rating.alias('t1'), on=F.col('t.userID') == F.col('t1.userID'), how='inner')\
  .drop(F.col('t1.userID'))\
  .join(df_mean_user_rating_year.alias('t2'), on=[F.col('t.userID') == F.col('t2.userID'),
                                      F.col('t.date_year') == F.col('t2.date_year')],
        how='inner')\
  .drop(F.col('t2.userID'))\
  .drop(F.col('t2.date_year'))

In [ ]:
res_join.show(5)

**union и unionAll**

Используются для объединения датафреймов с одинаковой структурой, используется union, так как unionAll с версии 2.0.0 более не используется

In [ ]:
df1 = df.filter(df.date_year == 2006)
df2 = df.filter(df.date_year != 2006)

In [ ]:
union_df = df1.union(df2)

print(df.count(), union_df.count())

Desclaimer: все по sql, надеюсь, помнят разницу между union и union all, когда union убирает дубликаты. Так вот pyspark ничего не удаляет, убрать дубликаты можно только через drop_duplicates, distinct

Также при union pyspark делает объединение по столбцам as is, не пытаясь понять, что в одном датафрейме нужный стобец на 1 позиции, а в другом он на 5. Для этого с версии 3.1 есть замечательный метод unionByName

**UDF - user defined functions**

из курса про rdd помним про map, тут тоже можно перегнать все в rdd и делать map, но можно и через udf. Стоит отметить, что при этом мы теряем возможность оптимизации и произодительность в dataframe, так как udf - black box для спарка.

Но зато эти функции переиспользуемы и их можно применять в sql запросах, как те же udf в oracle

In [ ]:
def udf_example(rating):
    rating = rating * 20
    return rating

In [3]:
from pyspark.sql.functions import udf

In [ ]:
my_udf = udf(lambda x: udf_example(x), DoubleType())

In [ ]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', my_udf(F.col('rating')))\
  .show(5)

Для тех, кто любит декораторы

In [ ]:
@udf(returnType=DoubleType()) 
def udf_example_decorator(rating):
    rating = rating * 20
    return rating

In [ ]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', udf_example_decorator(F.col('rating')))\
  .show(5)

Зарегистрируем функцию для будущих примеров с sql

In [ ]:
spark.udf.register("udf_example_decorator", udf_example_decorator)

**SQL**

Ну раз уж пошла такая тема, давайте рассмотрим, как можно сделать все при помощи любимого SQL

можно делать TempView и GlodalTempView, отличие в том, что обычный view будет жить, пока жива сессия спрака, а глобальная, пока жив sparkcontext

In [ ]:
df.createOrReplaceTempView('df')

In [ ]:
query = '''
select userID, movieID, rating, udf_example_decorator(rating) as rating_100
from
df
'''
spark.sql(query).show(5)

Ну и наш join

In [ ]:
df_mean_user_rating.createOrReplaceTempView('df_mean_user_rating')
df_mean_user_rating_year.createOrReplaceTempView('df_mean_user_rating_year')

In [ ]:
query = '''
select t.*, t1.avg_rating_all, t2.avg_rating_year
from
df t, df_mean_user_rating t1, df_mean_user_rating_year t2
where
    t.userID = t1.userID and
    t.userID = t2.userID and
    t.date_year = t2.date_year
'''
spark.sql(query).show(5)

**fill() и fillna()**

Оба метода идентичны, заполняют пропуски

In [4]:
import numpy as np

In [89]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5, None), (6, None),
     (None, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

А где пропуски?

In [90]:
df2.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+---+----+
| id|name|
+---+----+
|  1|   2|
+---+----+



In [91]:
df2.show()

+----+---------------+
|  id|           name|
+----+---------------+
|   2|   Michael Rose|
|   3|Robert Williams|
|   4|     Rames Rose|
|   5|           null|
|   6|           null|
|null|        Fred Tf|
+----+---------------+



In [ ]:
df2.show()

In [ ]:
df2.fillna({'id': 0}).show()

In [ ]:
df2.fillna({'id': 0, 'name': 'Unknown'}).show()

Аналогично

In [ ]:
df2.na.fill({'id': 0, 'name': 'Unknown'}).show()

Немного поговорим о том, как делаеть подвыборки

**sample и sampleBy**

Не забываем про возможную некоторую недетерминированность

In [5]:
import pyspark.sql.functions as F

In [10]:
trans_data = spark.read.parquet('/kaggle/input/transss/spark_transactions.parquet')
trans_data = trans_data.withColumn('target', F.when(F.col('IsFraud')=='No', 0).otherwise(1))

In [11]:
trans_data.count()

24386900

In [12]:
trans_data.select(F.mean(F.col('target'))).show()

+--------------------+
|         avg(target)|
+--------------------+
|0.001220204290008...|
+--------------------+



In [13]:
trans_data.select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+--------+
|target|   count|
+------+--------+
|     1|   29757|
|     0|24357143|
+------+--------+



In [31]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.1, seed=3)
print(trans_data_simple.count())
trans_data_simple.select(F.mean(F.col('target'))).show()

2438288


+--------------------+
|         avg(target)|
+--------------------+
|0.001227910730807...|
+--------------------+



Стратификация

Тут важно понимать, что это не scikit-learn и стратификация предполагает, что вы по какому-то полю можете выбрать определенную долю наблюдений по его значениям

In [32]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+-----+
|target|count|
+------+-----+
|     1|29757|
+------+-----+



Не указали какой-то ключ - его доля будет 0

In [33]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0, 0: 0.1}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+-------+
|target|  count|
+------+-------+
|     1|  29757|
|     0|2436011|
+------+-------+



Оконные функции **F.func().over(Window.partitionBy().orderBy())**

In [6]:
from pyspark.sql.window import Window

In [35]:
trans_data_simple.select('User', 'Card', 'Year', 'Month', 'Day')\
.withColumn('rn_1', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc())))\
.withColumn('rn_2', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc(), F.col('Month').desc())))\
.withColumn('mean', F.mean('Day').over(Window.partitionBy('User', 'Card', 'Month')))\
.withColumn('lag', F.lag('Day').over(Window.partitionBy('User', 'Card', 'Month').orderBy(F.col('Year').asc(),
                                                                                         F.col('Month').desc(),
                                                                                         F.col('Day').desc())))\
.show()

+----+----+----+-----+---+----+----+----+----+
|User|Card|Year|Month|Day|rn_1|rn_2|mean| lag|
+----+----+----+-----+---+----+----+----+----+
|   1|   0|2003|    9| 22|   1|   1|22.0|null|
|   1|   1|2012|    1| 26|  18|  34|14.3|null|
|   1|   1|2012|    1| 18|  17|  33|14.3|  26|
|   1|   1|2012|    1|  2|  16|  32|14.3|  18|
|   1|   1|2013|    1| 17|  35|  57|14.3|   2|
|   1|   1|2014|    1|  6|  59|  73|14.3|  17|
|   1|   1|2014|    1|  1|  58|  72|14.3|   6|
|   1|   1|2015|    1| 23|  75|  95|14.3|   1|
|   1|   1|2015|    1| 21|  74|  94|14.3|  23|
|   1|   1|2016|    1|  3|  97| 115|14.3|  21|
|   1|   1|2016|    1|  2|  96| 114|14.3|   3|
|   1|   1|2017|    1| 24| 117| 142|14.3|   2|
|   1|   1|2017|    1|  3| 116| 141|14.3|  24|
|   1|   1|2018|    1| 27| 145| 168|14.3|   3|
|   1|   1|2018|    1| 22| 144| 167|14.3|  27|
|   1|   1|2018|    1|  7| 143| 166|14.3|  22|
|   1|   1|2019|    1| 17| 170| 202|14.3|   7|
|   1|   1|2019|    1|  1| 169| 201|14.3|  17|
|   1|   1|20

Сборка последовательностей через **collect_list()**

In [36]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)
 |-- target: integer (nullable = false)



In [37]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Amount', 'Day').alias('sequence'))

In [38]:
trans_data_simple_seq.show()

+----+----+-------------+
|User|Card|     sequence|
+----+----+-------------+
|   0|   0|  {$38.48, 1}|
|   0|   0| {$104.71, 3}|
|   0|   0|  {$61.72, 5}|
|   0|   0|  {$76.07, 5}|
|   0|   0|  {$76.57, 8}|
|   0|   0| {$52.52, 12}|
|   0|   0| {$12.76, 14}|
|   0|   0| {$55.41, 28}|
|   0|   0|{$153.36, 28}|
|   0|   0|{$165.96, 29}|
|   0|   0|  {$34.89, 1}|
|   0|   0| {$102.74, 3}|
|   0|   0| {$133.96, 7}|
|   0|   0|  {$53.09, 9}|
|   0|   0|{$142.06, 10}|
|   0|   0| {$37.69, 11}|
|   0|   0|{$168.96, 12}|
|   0|   0|{$100.75, 14}|
|   0|   0| {$24.02, 19}|
|   0|   0| {$41.02, 20}|
+----+----+-------------+
only showing top 20 rows



In [39]:
trans_data_simple_seq = trans_data_simple_seq.groupBy('User', 'Card')\
                                             .agg(F.collect_list('sequence').alias('sequence'))

In [40]:
trans_data_simple_seq.show(truncate=50)

+----+----+--------------------------------------------------+
|User|Card|                                          sequence|
+----+----+--------------------------------------------------+
|   1|   0|                                   [{$177.59, 22}]|
|   1|   1|[{$109.28, 5}, {$27.47, 21}, {$0.66, 31}, {$107...|
|   1|   3|[{$22.49, 15}, {$37.19, 2}, {$43.31, 16}, {$73....|
|   3|   0|[{$214.00, 15}, {$400.00, 15}, {$69.33, 25}, {$...|
|   4|   0|[{$163.02, 3}, {$111.54, 16}, {$15.70, 20}, {$8...|
|   5|   1|[{$26.33, 3}, {$27.10, 4}, {$19.22, 18}, {$60.0...|
|  13|   0|[{$200.22, 21}, {$16.98, 24}, {$17.90, 11}, {$2...|
|  13|   1|[{$20.85, 3}, {$7.64, 3}, {$100.00, 10}, {$80.0...|
|  15|   0|[{$60.00, 4}, {$9.39, 7}, {$24.23, 17}, {$8.83,...|
|  16|   4|[{$57.01, 4}, {$60.93, 17}, {$66.74, 27}, {$10....|
|  17|   3|                                   [{$117.38, 24}]|
|  18|   0|[{$200.00, 6}, {$3.31, 7}, {$100.00, 7}, {$2.74...|
|  19|   2|[{$147.42, 7}, {$72.29, 8}, {$20.31, 10}, {$

А что с этим делать дальше? Можно перейти к RDD и построчно обрабатывать

In [41]:
row = trans_data_simple_seq.rdd.take(1)

In [42]:
row

[Row(User=1, Card=0, sequence=[Row(Amount='$177.59', Day=22)])]

In [43]:
row[0]['sequence']

[Row(Amount='$177.59', Day=22)]

**Домашнее задание**

Куда же без домашки, верно?

Есть данные по транзакциям клиентов, ваша задача состоит в анализе этих данных и подготовки к структуре, которая похожа на ту структуру, которая сейчас часто нами используется при построении моделей на транзакциях + промежуточные задания.

Не забудьте делать всякие show после каждого задания, чтобы было видно результат

**Файл spark_transactions.parquet можете забрать в папке с записями лекций**

**Важно**
В домашнем задании старайтесь использовать максимально dataframe api, а не sql запросы.

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark.sql.functions import mean, countDistinct, split, when, substring, length

In [4]:
trans_data = spark.read.parquet('/kaggle/input/transss/spark_transactions.parquet')
trans_data.show(5)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|Monterey Park|           CA|91754.0|5651|     No|
|   0|   0|2002|    9|  3|06:23|$104.71|Swipe Transaction|     La Verne|           CA|91750.0|5912|     No|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
only showing top 5 rows



In [5]:
trans_data.count() 

24386900

In [6]:
trans_data.show(5)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|Monterey Park|           CA|91754.0|5651|     No|
|   0|   0|2002|    9|  3|06:23|$104.71|Swipe Transaction|     La Verne|           CA|91750.0|5912|     No|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
only showing top 5 rows



Посмотрим на схему данных

In [7]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)



Сколько в среднем транзакций у пользователя

In [8]:
trans_data.groupBy('User').count().select(mean('count')).show()

+----------+
|avg(count)|
+----------+
|  12193.45|
+----------+



In [9]:
trans_data.show(10)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|Monterey Park|           CA|91754.0|5651|     No|
|   0|   0|2002|    9|  3|06:23|$104.71|Swipe Transaction|     La Verne|           CA|91750.0|5912|     No|
|   0|   0|2002|    9|  3|13:53| $86.19|Swipe Transaction|Monterey Park|           CA|91755.0|5970|     No|
|   0|   0|2002|    9|  4|05

Сколько карт у пользователей в среднем

In [10]:
trans_data.groupBy('User').agg(countDistinct('Card')).select(mean('count(Card)')).show()

+----------------+
|avg(count(Card))|
+----------------+
|          3.0695|
+----------------+



Немного обработаем данные: Amount в float, из Time вытянем час транзакции и удалим исходный Time, Zip  к типу int

In [11]:
trans_data = trans_data.withColumn("Amount", expr("substring(Amount, 2, length(Amount)-1)".format(n=1)))\
.withColumn("Amount", F.col("Amount").cast("float"))\
.withColumn("Zip", F.col("Zip").cast("int"))\
.withColumn("Time", split(F.col("Time"), ':').getItem(0))

In [12]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: float (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)



Посчитайте количество транзакций по годам, учитывая только те транзакции, объем которых был больше 100

In [13]:
trans_data.filter(trans_data.Amount > 100).groupBy('Year').count().show(5)

+----+------+
|Year| count|
+----+------+
|2007|121489|
|2014|179492|
|2012|174439|
|1991|   266|
|2016|182742|
+----+------+
only showing top 5 rows



Определите, есть ли пропуски в данных по каждому столбцу

In [92]:
trans_data.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in trans_data.columns]).show()

+----+----+----+-----+---+----+------+-------+------------+-------------+-------+---+-------+
|User|Card|Year|Month|Day|Time|Amount|UseChip|MerchantCity|MerchantState|    Zip|MCC|IsFraud|
+----+----+----+-----+---+----+------+-------+------------+-------------+-------+---+-------+
|   0|   0|   0|    0|  0|   0|     0|      0|           0|      2720821|2878135|  0|      0|
+----+----+----+-----+---+----+------+-------+------------+-------------+-------+---+-------+



Заполните пропуски исходя из типа данных

При помощи оконных функций для каждого клиента рассчитайте средний размер транзакции, количество транзакций и последнюю по дате транзакцию.

In [14]:
w2 = Window.partitionBy("User").orderBy(F.col('Year').desc(), F.col('Month').desc(), F.col('Day').desc())

In [15]:
trans_data.withColumn("row", row_number().over(w2)) \
.withColumn('Avg_trans', F.mean('Amount').over(Window.partitionBy('User')))\
.withColumn('Count_trans', F.count('Amount').over(Window.partitionBy('User')))\
.filter(col("row") == 1)\
.select("User", 'Year', 'Month', 'Day', 'Avg_trans', 'Count_trans').show(5)

+----+----+-----+---+------------------+-----------+
|User|Year|Month|Day|         Avg_trans|Count_trans|
+----+----+-----+---+------------------+-----------+
|   0|2020|    2| 28| 81.29998946268704|      19963|
|   6|2020|    2| 28| 35.79467471044497|      28237|
|   7|2020|    2| 28| 53.51347442754244|        236|
|  19|2020|    2| 28| 44.06187398630507|      49584|
|  22|2020|    2| 28|30.810217510490546|      16458|
+----+----+-----+---+------------------+-----------+
only showing top 5 rows



Теперь самое время сгруппировать данные по каждому клиенту (можно использовать collect_list для сбора данных после агрегации)
Когда будете делать агрегацию, то возьмите только часть выборки, например, через sample, для всей выборки либо не хватит памяти, либо очень долго считать

In [92]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.03, seed=3)

In [93]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Year', 'Month','Day', 'Time', 'MCC', 'Amount').alias('sequence'))

In [94]:
trans_data_simple_seq.show(5)

+----+----+--------------------+
|User|Card|            sequence|
+----+----+--------------------+
|   0|   0|{2002, 9, 3, 06, ...|
|   0|   0|{2002, 9, 28, 13,...|
|   0|   0|{2002, 10, 7, 06,...|
|   0|   0|{2002, 10, 11, 06...|
|   0|   0|{2002, 10, 19, 06...|
+----+----+--------------------+
only showing top 5 rows



In [95]:
trans_seq = trans_data_simple_seq.groupBy("User", 'Card').agg(collect_list("sequence"))

In [28]:
row = trans_seq.select(trans_seq.columns[2:3]).collect()[0]

In [55]:
row[0]

[Row(Year=2011, Month=4, Day=7, Time='07', MCC=5655, Amount=107.98999786376953),
 Row(Year=2011, Month=4, Day=21, Time='21', MCC=5813, Amount=24.93000030517578),
 Row(Year=2011, Month=9, Day=3, Time='08', MCC=5541, Amount=70.0),
 Row(Year=2011, Month=11, Day=18, Time='10', MCC=5942, Amount=71.1500015258789),
 Row(Year=2011, Month=12, Day=28, Time='21', MCC=3771, Amount=366.0),
 Row(Year=2011, Month=12, Day=30, Time='21', MCC=3390, Amount=190.44000244140625),
 Row(Year=2012, Month=3, Day=29, Time='08', MCC=5541, Amount=98.0999984741211),
 Row(Year=2012, Month=7, Day=16, Time='07', MCC=5541, Amount=92.54000091552734),
 Row(Year=2012, Month=7, Day=16, Time='07', MCC=5541, Amount=60.0),
 Row(Year=2012, Month=9, Day=19, Time='06', MCC=4814, Amount=75.83000183105469),
 Row(Year=2012, Month=10, Day=14, Time='13', MCC=7538, Amount=55.869998931884766),
 Row(Year=2013, Month=2, Day=25, Time='18', MCC=3359, Amount=366.0),
 Row(Year=2013, Month=4, Day=14, Time='22', MCC=4829, Amount=100.0),
 Row(Y

In [31]:
from operator import itemgetter

In [52]:
newrow

[Row(Year=2011, Month=4, Day=7, Time='07', MCC=5655, Amount=107.98999786376953),
 Row(Year=2011, Month=4, Day=21, Time='21', MCC=5813, Amount=24.93000030517578),
 Row(Year=2011, Month=9, Day=3, Time='08', MCC=5541, Amount=70.0),
 Row(Year=2011, Month=11, Day=18, Time='10', MCC=5942, Amount=71.1500015258789),
 Row(Year=2011, Month=12, Day=28, Time='21', MCC=3771, Amount=366.0),
 Row(Year=2011, Month=12, Day=30, Time='21', MCC=3390, Amount=190.44000244140625),
 Row(Year=2012, Month=3, Day=29, Time='08', MCC=5541, Amount=98.0999984741211),
 Row(Year=2012, Month=7, Day=16, Time='07', MCC=5541, Amount=92.54000091552734),
 Row(Year=2012, Month=7, Day=16, Time='07', MCC=5541, Amount=60.0),
 Row(Year=2012, Month=9, Day=19, Time='06', MCC=4814, Amount=75.83000183105469),
 Row(Year=2012, Month=10, Day=14, Time='13', MCC=7538, Amount=55.869998931884766),
 Row(Year=2013, Month=2, Day=25, Time='18', MCC=3359, Amount=366.0),
 Row(Year=2013, Month=4, Day=14, Time='22', MCC=4829, Amount=100.0),
 Row(Y

In [38]:
newrow[0]

Row(Year=2011, Month=4, Day=7, Time='07', MCC=5655, Amount=107.98999786376953)

In [37]:
newrow[0]['Year']

2011

In [32]:
newrow = sorted(row[0], key=itemgetter('Year', 'Month', 'Day', 'Time')) 

In [23]:
trans_seq.show(1)

+----+----+----------------------+
|User|Card|collect_list(sequence)|
+----+----+----------------------+
|   1|   1|  [{2011, 4, 7, 07,...|
+----+----+----------------------+
only showing top 1 row



Напишите python функцию, которая возьмет данные после агрегации последовательностей, отсортирует их внутри по дате и времени и преобразует к формату python dict:
{'User': User,
'Card': Card,
'sequence':{
    'amount': [последовательность],
    'year': [последовательность],
    'month': [последовательность],
    'day': [последовательность],
    'time': [последовательность],
    'MCC': [последовательность]
}
}




Выведите как пример одну преобразованную запись, результаты сохраните на диск в через rdd pickle

In [96]:
name_seq = ['Year', 'Month','Day', 'Time', 'MCC', 'Amount']

def coll_dict(row):
    trans_dc = {}
    new_listr = sorted(row_value, key=itemgetter('Year', 'Month', 'Day', 'Time')) 
    
    for seq in row:
        for name in name_seq:
            if name not in  trans_dc:
                trans_dc[f'{name}'] = [seq[f'{name}']]
            else:
                trans_dc[f'{name}'].append(seq[f'{name}'])
        
    return(trans_dc)

In [82]:
from pyspark.sql.functions import udf

In [89]:
trans_seq.show(1)

+----+----+----------------------+
|User|Card|collect_list(sequence)|
+----+----+----------------------+
|   1|   1|  [{2011, 4, 7, 07,...|
+----+----+----------------------+
only showing top 1 row



In [88]:
trans_seq.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- collect_list(sequence): array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- Year: long (nullable = true)
 |    |    |-- Month: long (nullable = true)
 |    |    |-- Day: long (nullable = true)
 |    |    |-- Time: string (nullable = true)
 |    |    |-- MCC: long (nullable = true)
 |    |    |-- Amount: float (nullable = true)



In [83]:
from pyspark.sql.types import *
name_seq = ['Year', 'Month','Day', 'Time', 'MCC', 'Amount']
udf_result = StructType([
    StructField('Year', ArrayType(LongType())),
    StructField('Month', ArrayType(LongType())),
    StructField('Day', ArrayType(LongType())),
    StructField('Time', ArrayType(StringType())),
    StructField('MCC', ArrayType(LongType())),
    StructField('Amount', ArrayType(FloatType()))
])

In [ ]:
my_udf = udf(lambda x: coll_dict(x), udf_result)

In [99]:
trans_struc = trans_seq.withColumn('Sequence', my_udf(F.col('collect_list(sequence)'))).drop('collect_list(sequence)')

In [112]:
result = trans_struc.select(F.struct('User', 'Card', 'Sequence'))

In [118]:
result.first()

Row(struct(User, Card, Sequence)=Row(User=1, Card=1, Sequence=Row(Year=[2011, 2011, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016, 2016, 2016, 2017, 2017, 2017, 2017, 2017, 2017, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2020], Month=[4, 4, 9, 11, 12, 12, 3, 7, 7, 9, 10, 2, 4, 5, 5, 6, 6, 8, 9, 10, 10, 1, 2, 5, 1, 4, 5, 10, 10, 1, 1, 10, 11, 12, 12, 2, 4, 6, 9, 9, 11, 1, 2, 4, 6, 6, 6, 7, 9, 10, 12, 5, 5, 5, 6, 7, 10, 11, 2], Day=[7, 21, 3, 18, 28, 30, 29, 16, 16, 19, 14, 25, 14, 8, 20, 12, 25, 9, 16, 2, 10, 6, 27, 14, 21, 27, 10, 3, 16, 2, 3, 10, 17, 5, 31, 11, 26, 3, 23, 25, 14, 7, 18, 9, 4, 6, 30, 2, 9, 30, 23, 1, 2, 24, 13, 2, 2, 3, 5], Time=['07', '21', '08', '10', '21', '21', '08', '07', '07', '06', '13', '18', '22', '05', '06', '17', '18', '18', '06', '06', '22', '06', '10', '08', '0